<a href="https://colab.research.google.com/github/sherlock-beep/tf_learn/blob/main/tf_learn5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -q -U keras-tuner

In [11]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

In [12]:
#加载数据集
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()
#对图片数据像素进行归一化
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0



In [15]:
from threading import active_count
#定义模型
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28,28)))
  #设置unit超参数，选择隐藏层的units(between 32-512)
  hp_units = hp.Int('units',min_value=32,max_value=512,step=32)
  model.add(keras.layers.Dense(units=hp_units,activation='relu'))
  model.add(keras.layers.Dense(10))
  #设置学习率超参数
  hp_learning_rate = hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model


In [16]:
#实例化调节器以执行超调。
#Keras Tuner 提供了四种调节器：
#RandomSearch、Hyperband、BayesianOptimization 和 Sklearn。

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [17]:
#创建回调以在验证损失达到特定值后提前停止训练。

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


In [18]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 07s]
val_accuracy: 0.8540833592414856

Best val_accuracy So Far: 0.8880000114440918
Total elapsed time: 00h 18m 03s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 512 and the optimal learning rate for the optimizer
is 0.001.



In [19]:
#使用获取的超参数获取最佳周期数

model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 10s 7ms/step - loss: 0.4927 - accuracy: 0.8259 - val_loss: 0.4025 - val_accuracy: 0.8528
Epoch 2/50
1500/1500 [==============================] - 10s 6ms/step - loss: 0.3705 - accuracy: 0.8650 - val_loss: 0.3508 - val_accuracy: 0.8701
Epoch 3/50
1500/1500 [==============================] - 12s 8ms/step - loss: 0.3306 - accuracy: 0.8780 - val_loss: 0.3335 - val_accuracy: 0.8804
Epoch 4/50
1500/1500 [==============================] - 10s 7ms/step - loss: 0.3052 - accuracy: 0.8875 - val_loss: 0.3344 - val_accuracy: 0.8802
Epoch 5/50
1500/1500 [==============================] - 10s 7ms/step - loss: 0.2875 - accuracy: 0.8924 - val_loss: 0.3342 - val_accuracy: 0.8812
Epoch 6/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2700 - accuracy: 0.8996 - val_loss: 0.3232 - val_accuracy: 0.8856
Epoch 7/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2574 - accuracy: 0.9045 - val_loss: 0.3863 - val_accu

In [21]:
# 重新实例化超模型并使用上面的最佳周期数对其进行训练。
hypermodel = tuner.hypermodel.build(best_hps)

hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/43
1500/1500 [==============================] - 10s 6ms/step - loss: 0.4906 - accuracy: 0.8255 - val_loss: 0.3983 - val_accuracy: 0.8573
Epoch 2/43
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3684 - accuracy: 0.8668 - val_loss: 0.3584 - val_accuracy: 0.8700
Epoch 3/43
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3311 - accuracy: 0.8775 - val_loss: 0.3641 - val_accuracy: 0.8647
Epoch 4/43
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3052 - accuracy: 0.8868 - val_loss: 0.3294 - val_accuracy: 0.8840
Epoch 5/43
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2851 - accuracy: 0.8935 - val_loss: 0.3234 - val_accuracy: 0.8858
Epoch 6/43
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2709 - accuracy: 0.8986 - val_loss: 0.3435 - val_accuracy: 0.8784
Epoch 7/43
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2569 - accuracy: 0.9027 - val_loss: 0.3403 - val_accuracy

In [22]:
#在test-dataset上评估超模型
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 4ms/step - loss: 0.5189 - accuracy: 0.8923
[test loss, test accuracy]: [0.5189301371574402, 0.892300009727478]


In [20]:
print('a')

a
